# Simply upload congressional-seating-chart.svg onto the local files and run all the cells to generate the graph!


In [2]:
!pip install requests beautifulsoup4 --quiet
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pprint as pprint
import json
from IPython.display import display, SVG, HTML, Javascript

In [10]:
def get_member_votes(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")

    table = soup.find('table', class_='vote-list stats')

    rows = table.find_all("tr")

    year = url.split("/")[5].split("-")[1]

    table_data = []

    for row in rows[1:]:
        cells = row.find_all("td")

        representative, party, district, vote, opensecrets_url = None, None, None, None, None

        if cells and len(cells) >= 4:
          vote = cells[0].text.strip()
          district = cells[1].text.strip()
          party = cells[2].text.strip()
          representative = cells[3].text.strip()

          name_parts = representative.split(",")
          if len(name_parts) == 2:
            first_name = name_parts[1].strip()
            last_name = name_parts[0]
            representative = f"{first_name} {last_name}"
            opensecrets_url = f"https://www.opensecrets.org/members-of-congress/search?q={last_name}"

        table_data.append([representative, party, district, vote, opensecrets_url])

    df = pd.DataFrame(table_data, columns=["Representative", "Party", "District", "Vote", "opensecrets_url"])
    return df

Paste desired House vote from https://www.govtrack.us/congress/votes. Full House votes should have around 435 representatives and smaller votes and motions will not render as well. Placeholder url is the vote for H.R. 23: Illegitimate Court Counteraction Act.

In [11]:
url = "https://www.govtrack.us/congress/votes/119-2025/h7"

In [12]:
df = get_member_votes(url)
df.sort_values(by=["Party", "District"], ascending=True, inplace=True)
json_data = df.to_json(orient='records')
json_data

'[{"Representative":"Shomari Figures","Party":"D","District":"AL\\u00a0\\u2013\\u00a02","Vote":"Yea","opensecrets_url":"https:\\/\\/www.opensecrets.org\\/members-of-congress\\/search?q=Figures"},{"Representative":"Terri Sewell","Party":"D","District":"AL\\u00a0\\u2013\\u00a07","Vote":"Nay","opensecrets_url":"https:\\/\\/www.opensecrets.org\\/members-of-congress\\/search?q=Sewell"},{"Representative":"Yassamin Ansari","Party":"D","District":"AZ\\u00a0\\u2013\\u00a03","Vote":"Yea","opensecrets_url":"https:\\/\\/www.opensecrets.org\\/members-of-congress\\/search?q=Ansari"},{"Representative":"Greg Stanton","Party":"D","District":"AZ\\u00a0\\u2013\\u00a04","Vote":"No Vote","opensecrets_url":"https:\\/\\/www.opensecrets.org\\/members-of-congress\\/search?q=Stanton"},{"Representative":"Ra\\u00fal Grijalva","Party":"D","District":"AZ\\u00a0\\u2013\\u00a07","Vote":"No Vote","opensecrets_url":"https:\\/\\/www.opensecrets.org\\/members-of-congress\\/search?q=Grijalva"},{"Representative":"Mark DeSa

# Final cell to run. View output at fullscreen and interact.
Click on a circle to search up the corresponding respresentative's campaign finance profiles on OpenSecrets. Notes: Some profiles won't be found because OpenSecrets' search is not really robust and I couldn't webscrape the exact profiles due to restrictions; Older votes will also not return profiles as OpenSecrets only has data from the 105th Congress and onwards.


In [17]:
# Read the SVG file
with open('congressional-seating-chart.svg', 'r') as file:
    svg_content = file.read()

# Create an HTML block with embedded D3.js script
html_content = f"""
<div id="svg-container" style="background-color: white; width: 1300px;">
    {svg_content}
</div>

<script src="https://d3js.org/d3.v7.min.js"></script>
<script>
    // Pass voting data to the script
    const data = {json_data};

    // Parse json_data as an Array
    const dataString = JSON.stringify(data);
    const array = JSON.parse(dataString);

    // Select the SVG using D3
    var svg = d3.select("#svg-container svg")
      .attr("width", 1250)
      .attr("height", 600);

    // Color scale for parties
    const color = d3.scaleOrdinal()
      .domain(["D", "R"])
      .range(["#00AEF3", "#E81B23"])
      .unknown("#9932CC");

    // Function to determine fill color based on vote
    function fill(d) {{
      if (d.Vote === "Yea") {{
        return color(d.Party);
        }} else if (d.Vote === "Nay") {{
        return "white";
      }} else {{
        return "black";
      }}
    }};

    // Data for the legend
    keys = [
        {{ index: 0, Representative: "Independent and Voted Yea", Party: "I", Vote: "Yea" }},
        {{ index: 1, Representative: "Democrat and Voted Nay", Party: "D", Vote: "Nay" }},
        {{ index: 2, Representative: "Republican and Did Not Vote", Party: "R", Vote: "Did Not Vote"}}
    ];

    // Function to create the legend
    function generateLegend(container) {{
      const titlePadding = 30;  // padding between title and entries
      const entrySpacing = 30;  // spacing between legend entries
      const entryRadius = 5;    // radius of legend entry marks
      const labelOffset = 30;    // additional horizontal offset of text labels
      const baselineOffset = 4; // text baseline offset, depends on radius and font size

      const title = container.append('text')
        .attr('x', 0)
        .attr('y', 0)
        .attr('fill', 'black')
        .attr('font-family', 'Helvetica Neue, Arial')
        .attr('font-weight', 'bold')
        .attr('font-size', '22px')
        .text('Legend');

      const entries = container.selectAll('g')
        .data(keys)
        .join('g')
        .attr('transform', d => `translate(0, ${{titlePadding + d.index * entrySpacing}})`);

      const symbols = entries.append('circle')
        .attr('cx', 17) // <-- offset symbol x-position by radius
        .attr('r', 10)
        .attr("stroke", d => color(d.Party))
        .attr("stroke-width", 4)
        .attr("fill", d => fill(d));

      const labels = entries.append('text')
        .attr('x', 2 * entryRadius + labelOffset) // <-- place labels to the left of symbols
        .attr('y', baselineOffset) // <-- adjust label y-position for proper alignment
        .attr('fill', 'black')
        .attr('font-family', 'Helvetica Neue, Arial')
        .attr('font-size', '18px')
        .style('user-select', 'none') // <-- disallow selectable text
        .text(d => d.Representative);
      }};

      function generateTooltip(container) {{
        const title = container.append('text')
          .attr('x', 0)
          .attr('y', 0)
          .attr('fill', 'black')
          .attr('font-family', 'Helvetica Neue, Arial')
          .attr('font-size', '28px')
          .attr('class', 'Description')
      }};

      // Data labeling of the circles
      const votes = svg
        .selectAll("circle")
        .data(array)
        .join("circle")
        .attr("stroke", d => color(d.Party))
        .attr("stroke-width", 4)
        .attr("fill", d => fill(d))
        .attr("transform", "translate(40,20)");

      // Generate the legend
      const legend = svg.append('g')
        .attr('transform', 'translate(0, 20)')
        .call(generateLegend);

      // Generate the tooltip
      const tooltip = svg.append('g')
        .attr('transform', 'translate(800, 30)')
        .call(generateTooltip);

      // Interaction with the circles
      votes
        .on('mouseover', function(event, d) {{
          d3.select(this).attr('stroke-width', 8);

          const text = d.Representative+', '+d.District;
          tooltip.select('.Description').text(text);

          // tooltip.style('display', 'block');
        }})
        .on('mouseout', function() {{
          d3.select(this).attr('stroke-width', 4);
          // tooltip.style('display', 'none');
        }})
        .on('click', function(event,d){{
          if (d.opensecrets_url) {{
            window.open(d.opensecrets_url, '_blank');
          }}
        }});
</script>
"""

display(HTML(html_content))